# Results

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
width=20
height=3
#plt.rcParams["figure.figsize"] = (width,height)
#sns.set(rc={'figure.figsize':(width,height)})

import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from d2e2f.visualization.visualize import plot_map, plot_trips
from myst_nb import glue
from jb_helpers import df_to_myst

In [ ]:
exclude_ships = catalog.load("params:ssrs.exclude_ships")
statistics_summary = catalog.load("ssrs.statistics_summary")
statistics_moving_summary = catalog.load("ssrs.statistics_moving_summary")
statistics_running_summary = catalog.load("ssrs.statistics_running_summary")
data_selected = catalog.load("ssrs.data_selected")
for key in exclude_ships:
    data_selected.pop(key)
    
data_running = catalog.load("ssrs.data_running")
for key in exclude_ships:
    data_running.pop(key)

models = catalog.load("ssrs.model_running")
    
statistics_summary.drop(index=exclude_ships, inplace=True)
statistics_moving_summary.drop(index=exclude_ships, inplace=True)
statistics_running_summary.drop(index=exclude_ships, inplace=True)

statistics_all = {
    'all' : statistics_summary,
    'moving' : statistics_moving_summary,
    'running' : statistics_running_summary,

}

## Statistics

In [ ]:
interesting = ['ship','consumption','consumption per hour', 'consumption per nautical mile', 'distance', 'sog', 'time']

statistics_summary_tables = {}
for item, statistics_summary in statistics_all.items():

    statistics_summary['sog'] = statistics_summary['sog']*3.6/1.852
    statistics_summary['ship'] = [key.split('_')[0] for key in statistics_summary.index]
    statistics_summary_table = statistics_summary[interesting].copy()
    statistics_summary_table['time'] = statistics_summary_table['time']/3600
    statistics_summary_table['distance'] = statistics_summary_table['distance']/1852
    units={
        'consumption' : 'm3',
        'consumption per hour' : 'l/h',
        'consumption per nautical mile' : 'l/NM',
        'distance':'NM', 
        'sog': 'kts',
        'time' : 'h',
    
    }
    
    renames = {key:f"{key} [{value}]" for key, value in units.items()}
    statistics_summary_table.rename(columns=renames, inplace=True)
    statistics_summary_tables[item] = statistics_summary_table
    
    print(df_to_myst(statistics_summary_table.round(decimals=1), 
                         title=f'Statistics {item}', 
                         name=f"statistics_{item}", include_index=False))

for item, statistics_summary in statistics_summary_tables.items():
    display(statistics_summary.round(decimals=1))

Data was collected during one month time for two vessels SSRS15-01 (01) and SSRS15-03 (03). The total fuel consumption and sailed distance are shown in {numref}`statistics_all` and {numref}`bars_all`. The fuel consumption during this period is very similar for both vessels. Majority of the time the ships are not moving, so that the average speed over ground (sog) is very small.

Data points where the ships are moving (sog > 0) have been filtered out and corresponding statistics are shown in {numref}`statistics_moving` and {numref}`bars_moving`. The average speed for 01 is now much higher than 03, which also increases the consumption per hour. Note that the consumption per nautical mile (CNM) is much lower for 01.

Data points where the ships are running (sog > 2 kts) have been filtered out and corresponding statistics are shown in {numref}`statistics_running` and {numref}`bars_running`. Note that 03 has only 7 hours of this data.


```{list-table} Statistics all
:header-rows: 1
:name: statistics_all
* - ship
  - consumption [m3]
  - consumption per hour [l/h]
  - consumption per nautical mile [l/NM]
  - distance [NM]
  - sog [kts]
  - time [h]
* - SSRS15-01
  - 7.7
  - 20.1
  - 11.6
  - 660.3
  - 1.7
  - 380.9
* - SSRS15-03
  - 7.0
  - 15.0
  - 33.3
  - 210.8
  - 0.4
  - 468.8

```


```{list-table} Statistics moving
:header-rows: 1
:name: statistics_moving
* - ship
  - consumption [m3]
  - consumption per hour [l/h]
  - consumption per nautical mile [l/NM]
  - distance [NM]
  - sog [kts]
  - time [h]
* - SSRS15-01
  - 4.6
  - 62.6
  - 7.0
  - 649.0
  - 8.9
  - 72.9
* - SSRS15-03
  - 4.0
  - 17.7
  - 20.4
  - 197.4
  - 0.9
  - 227.9

```


```{list-table} Statistics running
:header-rows: 1
:name: statistics_running
* - ship
  - consumption [m3]
  - consumption per hour [l/h]
  - consumption per nautical mile [l/NM]
  - distance [NM]
  - sog [kts]
  - time [h]
* - SSRS15-01
  - 4.0
  - 107.2
  - 6.2
  - 638.9
  - 17.2
  - 37.2
* - SSRS15-03
  - 1.2
  - 164.1
  - 7.6
  - 153.7
  - 21.5
  - 7.2

```




```{glue:figure} bars_all
:figwidth: 1000px
:name: "bars_all"

Comparison between ships using all data
```

```{glue:figure} bars_moving
:figwidth: 1000px
:name: "bars_moving"

Comparison between ships using data where ships are moving
```

```{glue:figure} bars_running
:figwidth: 1000px
:name: "bars_running"

Comparison between ships using data where ships are running.
```

In [ ]:
for item, statistics_summary_table in statistics_summary_tables.items():
    #df_ = statistics_summary_table.select_dtypes(exclude='object').transpose()
    df_ = statistics_summary_table.set_index('ship', drop=True).transpose()
    df_ = df_.divide(df_.sum(axis=1), axis=0)
    
    fig,ax=plt.subplots()
    df_.plot.barh(ax=ax,stacked=True)
    glue(f"bars_{item}", fig, display=False);

In [ ]:
data = pd.DataFrame()
for ship, loader in data_selected.items():
    data = data.append(loader(), ignore_index=True)
    
data['sog_kt'] = data['sog']*3.6/1.852
data['fuel_per_m'] = data['fuel_rate']/data['sog']*1000/3600*1852
data['sog_kt_round'] = 2*(1/2*data['sog_kt']).round(decimals=0)
df_speed_table = data.groupby(by='sog_kt_round').mean()
df_speed_table_std = data.groupby(by='sog_kt_round').std()

If the speeds are divided into equaly spaced (2 kts) bins, mean values and standard deviation can be calculated for each bin to see how the consumption changes with speed which can be seen in {numref}`mean_consumption_sog` and {numref}`speed_table`.

```{glue:figure} mean_consumption_sog
:figwidth: 1000px
:name: "mean_consumption_sog"

Mean consumption per nautical mile for the whole speed range with data for both vessels. 
```

In [ ]:
fig,ax=plt.subplots()
df_speed_table.plot(x='sog_kt', y='fuel_per_m', label='mean', style='k-', ax=ax)
y1 = (df_speed_table - df_speed_table_std)['fuel_per_m']
y2 = (df_speed_table + df_speed_table_std)['fuel_per_m']


ax.fill_between(df_speed_table['sog_kt'], y1, y2, zorder=-10, color='lightgray', label='+/-std')
ax.set_ylim(0,15.0)
ax.legend()
ax.grid(True)
ax.set_ylabel('CNM [l/NM]')
ax.set_xlabel('sog [kts]')

glue("mean_consumption_sog",fig, display=False)


In [ ]:
speed_table = df_speed_table[['sog_kt','rpm','fuel_rate','fuel_per_m']].copy()
speed_table['sog_kt'] = speed_table['sog_kt'].round(decimals=1)
speed_table['rpm'] = speed_table['rpm'].round(decimals=0).astype(int)
speed_table['fuel_rate'] = (1000*speed_table['fuel_rate']).round(decimals=1)
speed_table['fuel_per_m'] = (speed_table['fuel_per_m']).round(decimals=1)

renames['fuel_rate'] = 'fuel rate [l/h]'
renames['fuel_per_m'] = 'consumption per nautical mile [l/NM]'

speed_table.rename(columns=renames, inplace=True)

print(df_to_myst(speed_table, 
                 title=f'Speed table', 
                 name=f"speed_table", include_index=False))

speed_table

```{list-table} Speed table
:header-rows: 1
:name: speed_table
* - sog_kt
  - rpm
  - fuel rate [l/h]
  - consumption per nautical mile [l/NM]
* - 0.1
  - 15.0
  - 11.8
  - 265.5
* - 1.7
  - 616.0
  - 17.7
  - 11.0
* - 4.1
  - 687.0
  - 22.7
  - 5.7
* - 6.1
  - 722.0
  - 27.6
  - 4.6
* - 7.9
  - 929.0
  - 44.1
  - 5.5
* - 9.8
  - 1203.0
  - 76.2
  - 7.8
* - 11.9
  - 1346.0
  - 93.5
  - 7.9
* - 14.0
  - 1395.0
  - 109.7
  - 7.9
* - 16.0
  - 1436.0
  - 118.7
  - 7.4
* - 18.0
  - 1481.0
  - 126.7
  - 7.0
* - 20.1
  - 1542.0
  - 134.9
  - 6.7
* - 22.0
  - 1580.0
  - 146.5
  - 6.7
* - 24.1
  - 1632.0
  - 157.0
  - 6.5
* - 26.0
  - 1673.0
  - 170.5
  - 6.6
* - 28.0
  - 1716.0
  - 178.5
  - 6.4
* - 29.9
  - 1755.0
  - 197.2
  - 6.6
* - 32.0
  - 1804.0
  - 216.2
  - 6.8
* - 34.1
  - 1857.0
  - 227.2
  - 6.7
* - 36.0
  - 1886.0
  - 239.2
  - 6.6
* - 37.6
  - 1947.0
  - 256.3
  - 6.8

```

## Fuel consumption model

The fuel consumption data for the two vessels are shown in {numref}`data_SSRS15-01_322012079` and {numref}`data_SSRS15-03_322012059`.

```{glue:figure} data_SSRS15-01_322012079
:figwidth: 1000px
:name: "data_SSRS15-01_322012079"

Data SSRS15-01
```

```{glue:figure} data_SSRS15-03_322012059
:figwidth: 1000px
:name: "data_SSRS15-03_322012059"

Data SSRS15-03
```

In [ ]:
for ship, loader in data_running.items():
    data = loader()
    model = models[ship]()
    
    data['sog_kt'] = data['sog']*3.6/1.852
    X_pred = pd.DataFrame()
    X_pred['sog'] = np.linspace(data['sog'].min(), data['sog'].max(), 100)
    
    prediction = X_pred.copy()
    prediction['fuel_rate'] = model.predict(X_pred)
    prediction['fuel_per_m'] = prediction['fuel_rate']/X_pred['sog']*1000/3600*1852
    prediction['sog_kt'] = prediction['sog']*3.6/1.852
    
    fig,axes=plt.subplots(nrows=2)
    ax =axes[0]
    data['fuel_per_m'] = data['fuel_rate']/data['sog']*1000/3600*1852
    data.resample('10S').mean().plot(x='sog_kt', y='fuel_per_m', style='.', alpha=0.1, ax=ax, label='__no_label__')
    
    ax.set_ylim(data['fuel_per_m'].quantile(0.01), 
                data['fuel_per_m'].quantile(0.90))
    ax.plot([],'b.', label='data')
    
    prediction.plot(x='sog_kt', y='fuel_per_m', ax=ax, label='regression')
    ax.set_xticks(np.arange(0,int(np.ceil(data['sog_kt'].max())),2));
    ax.grid(True)
    ax.legend()
    ax.set_ylabel('CNM [L/NM]')
    ax.set_xlabel('Speed over ground [kts]')
    
    
    ax = axes[1]
    data['sog_kt'].hist(bins=50, density=True, ax=ax)
    ax.set_xticks(np.arange(0,int(np.ceil(data['sog_kt'].max())),2));
    ax.set_xlabel('sog [m/s]');
    ax.set_ylabel('Probability density')
    ax.set_xlabel('Speed over ground [kts]')
    
    name = f"data_{ship}"
    display(name)
    glue(name, fig, display=False)

## Fuel saving potential

In [ ]:
ship = 'SSRS15-01_322012079'
data = data_running[ship]()
data['sog_kt'] = data['sog']*3.6/1.852
data['fuel_per_m'] = data['fuel_rate']/data['sog']*1000/3600*1852
model = models['all']()

In [ ]:
good_speeds = [5,25,36]

In [ ]:
import scipy.interpolate
f = scipy.interpolate.interp1d([0] + good_speeds + [50], 
                               [good_speeds[0]] + good_speeds + [good_speeds[-1]], 
                               kind='nearest', 
                               bounds_error=False)
data['good_speed'] = f(data['sog_kt'].values)

A fuel saving scenario is simulated for 01. In this scenario all data points are divided into three speeds groups: 5, 25 and 36 kts (see {numref}`scenario`). The speed is changed to either 5, 25 or 36 kts depending on the group. The fuel consumption is predicted with the developed model covering the same distance for each group.

```{glue:figure} scenario
:figwidth: 1000px
:name: "scenario"

Fuel saving scenario using speeds: 5, 25 and 36 kts.
```

In [ ]:
fig,ax=plt.subplots()

X_pred = pd.DataFrame()
X_pred['sog'] = np.linspace(data['sog'].min(), data['sog'].max(), 100)

prediction = X_pred.copy()
prediction['fuel_rate'] = model.predict(X_pred)
prediction['fuel_per_m'] = prediction['fuel_rate']/X_pred['sog']*1000/3600*1852
prediction['sog_kt'] = prediction['sog']*3.6/1.852
prediction.plot(x='sog_kt', y='fuel_per_m', ax=ax, style='k-', label='regression')

for good_speed, group in data.groupby(by=['good_speed'], sort=False):
    
    plot = group.resample('10S').mean().plot(x='sog_kt', y='fuel_per_m', 
                                      style='.', alpha=0.1, ax=ax, label="__no_legend__")
    
    ax.plot([], '.', label=f"{good_speed} kts", color=plot.get_lines()[-1].get_color())
    
    sog = good_speed*1.852/3.6
    fuel_rate = model.predict([[sog]])
    fuel_per_m = fuel_rate/sog*1000/3600*1852
    
    ax.plot(good_speed, fuel_per_m,'o', color=ax.get_lines()[-1].get_color())
    
    ax.set_ylim(group['fuel_per_m'].quantile(0.01), 
                group['fuel_per_m'].quantile(0.90))
ax.legend(loc='lower right')
ax.grid(True)
ax.set_ylabel('CNM [L/NM]')
ax.set_xlabel('Speed over ground [m/s]')
glue("scenario",fig, display=False)

In [ ]:
consumptions = pd.Series()

for good_speed, group in data.groupby(by=['good_speed'], sort=False):
    
    sog = good_speed*1.852/3.6
    distance = scipy.integrate.simps(group["sog"], dx=1.0)  # [m]
    time = distance/sog
        
    fuel_rate = model.predict([[sog]])[0]
    consumption = fuel_rate/3600*time
    consumptions[good_speed] = consumption

In [ ]:
df_consumptions = {key: values.loc[ship,'consumption'] for key,values in statistics_all.items()}
df_consumptions['scenario'] = consumptions.sum()
df_consumptions = pd.Series(df_consumptions)
table = df_consumptions.to_frame(name='total fuel consumption [m3]').transpose().round(decimals=1)

print(df_to_myst(table, 
                 title=f'Fuel saving potential', 
                 name=f"total_fuel_consumption", include_index=True))

The predicted total fuel consumption for 01 in the scenario is shown together with the total fuel consumtion for all data, moving data and running data in {numref}`total_fuel_consumption`

```{list-table} Fuel saving potential
:header-rows: 1
:name: total_fuel_consumption
* -  
  - all
  - moving
  - running
  - scenario
* - total fuel consumption [m3]
  - 7.7
  - 4.6
  - 4.0
  - 3.9

```